In [66]:
import mysql.connector as connector
from mysql.connector.pooling import MySQLConnectionPool as p
from mysql.connector import Error
# database config
dbconfig = {
    'database':'little_lemon_db',
    'user':'root',
    'password':'rootroot'
}
# trying to create a pool
try:
    pool = p(pool_name = 'pool_b',
              pool_size = 2,
              **dbconfig)
    print('Connection pool is created with a name: {}'.format(pool.pool_name))
    print('Pool size is: {}'.format(pool.pool_size))
except Error as err:
    print('Error code: {}'.format(err.errno))
    print('Error message: {}'.format(err.msg))

Connection pool is created with a name: pool_b
Pool size is: 2


In [6]:
# trying to get connections from the pool to enter data to Bookings table
try:
    connection1 = pool.get_connection()
except:
    print('No more connections are available')
    print('Adding connections to the pool')
    connection1 = connector.connect(**dbconfig)
    pool.add_connection(connection1)
    print('A new connection is added to the pool')

insert_bookings="""
INSERT INTO Bookings (TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)
VALUES
(8, 'Anees','Java','18:00:00',6);"""

cursor = connection1.cursor()
cursor.execute(insert_bookings)
connection1.commit()

In [10]:
try:
    connection2 = pool.get_connection()
except:
    print('No more connections are available')
    print('Adding connections to the pool')
    connection2 = connector.connect(**dbconfig)
    pool.add_connection(connection2)
    print('A new connection is added to the pool')

insert_bookings="""
INSERT INTO Bookings (TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)
VALUES
(5, 'Bald','Vin','19:00:00',6);"""

cursor = connection2.cursor()
cursor.execute(insert_bookings)
connection2.commit()

No more connections are available
Adding connections to the pool
A new connection is added to the pool


In [12]:
try:
    connection3 = pool.get_connection()
except:
    print('No more connections are available')
    print('Adding connections to the pool')
    connection3 = connector.connect(**dbconfig)
    pool.add_connection(connection3)
    print('A new connection is added to the pool')

insert_bookings="""
INSERT INTO Bookings (TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)
VALUES
(12, 'Jay','Kon','18:30:00',6);"""

cursor = connection3.cursor()
cursor.execute(insert_bookings)
connection3.commit()

In [60]:
# returning connections to the pool
try:
    connection1.close()
except Error as err:
    print('Error code: {}'.format(err.errno))
    print('Error message: {}'.format(err.msg))

try:
    connection2.close()
except Error as err:
    print('Error code: {}'.format(err.errno))
    print('Error message: {}'.format(err.msg))
    
try:
    connection3.close()
except Error as err:
    print('Error code: {}'.format(err.errno))
    print('Error message: {}'.format(err.msg))

AttributeError: 'NoneType' object has no attribute 'reset_session'

In [59]:
# getting all the data needed for the report
connection = pool.get_connection()
cursor = connection.cursor()

sql_query1 = '''
select EmployeeID, Name
from Employees
where Role = 'Manager'
'''
cursor.execute(sql_query1)
results1 = cursor.fetchall()

sql_query2 = '''
select Name, Role
from Employees
order by Annual_Salary desc
limit 1
'''
cursor.execute(sql_query2)
results2 = cursor.fetchall()

sql_query3 = '''
select count(BookingID)
from Bookings
where hour(BookingSlot) between '18:00' and '20:00'
'''
cursor.execute(sql_query3)
results3 = cursor.fetchall()

sql_query4 = '''
select
    concat(b.GuestFirstName, ' ', b.GuestLastName) as GuestFullName,
    b.BookingID
from Bookings b left outer join Employees e on b.EmployeeID = e.EmployeeID
where e.Role = 'Receptionist'
order by BookingSlot asc 
'''
cursor.execute(sql_query4)
results4 = cursor.fetchall()

# composing report
for row in results1:
    print('Little Lemon manager is {}. His ID is {}.'.format(row[1], row[0]))
for row in results2:
    print('The highest paying employee of Little Lemon is {}. He/she is {}.'.format(row[0], row[1]))
for row in results3:
    print('The number of guests booked between 18:00 and 20:00 is {}.'.format(row[0]))
print('Here are the full names and BookingIDs of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot:')
for row in results4:
    print(row)

connection.close()

PoolError: Failed getting connection; pool exhausted

In [58]:
# composing basic sales report
connection = pool.get_connection()
cursor = connection.cursor()

stored_procedure_query = '''
create procedure BasicSalesReport()

begin

select
    round(sum(BillAmount), 2) as TotalSales,
    round(avg(BillAmount), 2) as AverageSales,
    round(min(BillAmount), 2) as MinBillPaid,
    max(BillAmount) as MaxBillPaid
from Orders;

end
'''
cursor.execute(stored_procedure_query)
cursor.callproc('BasicSalesReport')
results = next(cursor.stored_results())
dataset = results.fetchall()

for column_id in results:
    columns = [column[0] for column in column_id.description]

print(columns)
for row in dataset:
    print(row)

connection.close()

PoolError: Failed getting connection; pool exhausted

In [67]:
# composing report containing the next three upcoming bookings
connection = pool.get_connection()
cursor = connection.cursor(buffered=True)

sql_query = '''
select
    BookingSlot,
    concat(b.GuestFirstName, ' ', b.GuestLastName) as GuestFullName,
    concat('Assigned to: ', e.Name, ' [', e.EmployeeID, ']') as AssignedEmployee
from Bookings b left outer join Employees e on b.EmployeeId = e.EmployeeId
order by BookingSlot asc
limit 3
'''
cursor.execute(sql_query)
results = cursor.fetchall()
columns = cursor.column_names
print(columns)
for row in results:
    print(row)

connection.close()

('BookingSlot', 'GuestFullName', 'AssignedEmployee')
(datetime.timedelta(seconds=54000), 'Vanessa McCarthy', 'Assigned to: Giorgos Dioudis [3]')
(datetime.timedelta(seconds=63000), 'Marcos Romero', 'Assigned to: Fatma Kaya [4]')
(datetime.timedelta(seconds=64800), 'Anees Java', 'Assigned to: John Millar [6]')
